In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sentiment/Sentiment_Analysis.csv


In [2]:
df=pd.read_csv("/kaggle/input/sentiment/Sentiment_Analysis.csv",encoding='utf-8-sig')

In [3]:
df

,sentiment,text
0,1,"And here is the rap song ""African Warrior Quee..."
1,1,"We asked chatGPT ,\n\nHow to become a successf..."
2,0,When I finally manage to convince myself to pe...
3,0,Although a healthier alternative to your typic...
4,1,"In my latest test, I asked #ChatGPT to write m..."
...,...,...
79995,1,This movie just might make you cooooo. The fil...
79996,0,Stack Overflow temporarily bans answers from O...
79997,0,"<span class=""tiny""> Length:: 0:49 Mins<br /><b..."
79998,1,If it is true that sadomasochism is a two-side...


In [4]:
X_train=df['text']
y_train=df['sentiment']

In [5]:
y_train.value_counts()

sentiment
1    40000
0    40000
Name: count, dtype: int64

In [6]:
X_train

0        And here is the rap song "African Warrior Quee...
1        We asked chatGPT ,\n\nHow to become a successf...
2        When I finally manage to convince myself to pe...
3        Although a healthier alternative to your typic...
4        In my latest test, I asked #ChatGPT to write m...
                               ...                        
79995    This movie just might make you cooooo. The fil...
79996    Stack Overflow temporarily bans answers from O...
79997    <span class="tiny"> Length:: 0:49 Mins<br /><b...
79998    If it is true that sadomasochism is a two-side...
79999    I was just watching a Forensic Files marathon ...
Name: text, Length: 80000, dtype: object

In [7]:
import re
import emoji
def clean_for_tokenizer(text):
    text = str(text)
    text=text.lower()
    text = emoji.demojize(text)
    text = text.replace('\\n', ' ')              # remove newlines
    text = re.sub(r'http\S+', '', text)         # remove URLs
    text = re.sub(r'@\w+', '', text)            # remove @usernames
    text = re.sub(r'#', '', text)# remove hashtags symbol but keep word
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r"[^a-z_: ]+", " ", text)#remove numbers and symbols
    text = re.sub(r"\s+", " ", text).strip()#to remove extra spaces

    return text
    
X_train_clean = [clean_for_tokenizer(x) for x in X_train]
X_train_clean[:2]

['and here is the rap song african warrior queens for which chatgpt wrote the lyrics :brown_heart: yes amateur but beautiful : on ko :crossed_swords: link below :speaker_high_volume: sound on',
 'we asked chatgpt how to become a successful trader here is what it thinks chatgpt']

In [8]:
byte_sequences=[text.encode('utf-8') for text in X_train_clean] 

In [9]:
from collections import defaultdict
pair_counts = defaultdict(int)#default value integer value->0
positions = defaultdict(set)#defaould value->{}

def build_pair_stats(ids):
    pair_counts = defaultdict(int)
    positions = defaultdict(set)

    for i in range(len(ids) - 1):
        pair = (ids[i], ids[i + 1])
        pair_counts[pair] += 1
        positions[pair].add(i)

    return pair_counts, positions

# Flatten the list of lists into a single list
tokens = [item for sublist in byte_sequences for item in sublist]


In [10]:
def apply_merge(ids, pair, new_id, pair_counts, positions, heap):
    a, b = pair
    affected_positions = sorted(positions[pair], reverse=True)

    for i in affected_positions:
         # skip invalid indices
        if i >= len(ids) - 1:
            continue
        if ids[i] != a or ids[i+1] != b:
            continue


        # Remove left pair
        if i > 0:
            left = (ids[i-1], a)
            pair_counts[left] -= 1
            heapq.heappush(heap, (-pair_counts[left], left))
            positions[left].discard(i-1)

        # Remove right pair
        right_idx = i + 2
        if right_idx < len(ids):
            right = (b, ids[right_idx])
            pair_counts[right] -= 1
            heapq.heappush(heap, (-pair_counts[right], right))
            positions[right].discard(i+1)

        # Merge
        ids[i] = new_id
        ids.pop(i+1)

        # Add new left pair
        if i > 0:
            new_left = (ids[i-1], new_id)
            pair_counts[new_left] += 1
            heapq.heappush(heap, (-pair_counts[new_left], new_left))
            positions[new_left].add(i-1)

        # Add new right pair
        if i + 1 < len(ids):
            new_right = (new_id, ids[i+1])
            pair_counts[new_right] += 1
            heapq.heappush(heap, (-pair_counts[new_right], new_right))
            positions[new_right].add(i)

    # Remove merged pair
    del pair_counts[pair]
    del positions[pair]




In [11]:
tokens[:5]

[97, 110, 100, 32, 104]

In [12]:
print(len(tokens))

37387765


In [13]:
import heapq

In [14]:
vocab_size = 25000 # the desired final vocabulary size
num_merges = vocab_size - 256
 # copy so we don't destroy the original list
ids = list(tokens[:5_000_000])
  # 1M tokens

pair_counts, positions = build_pair_stats(ids)
merges={}
heap = [(-count, pair) for pair, count in pair_counts.items()]
heapq.heapify(heap)

for i in range(num_merges):
    found = False
    while heap:
        neg_count, pair = heapq.heappop(heap)
        if pair in pair_counts and pair_counts[pair] == -neg_count:
            found = True
            break

    if not found or -neg_count < 2:
        break


    new_id = 256 + i
    apply_merge(ids, pair, new_id, pair_counts, positions, heap)
    print(f"merged{pair} into {new_id}")

    merges[pair] = new_id
    if i % 100 == 0:
        heap = [(-c, p) for p, c in pair_counts.items()]
        heapq.heapify(heap)


merged(101, 32) into 256
merged(32, 116) into 257
merged(115, 32) into 258
merged(116, 104) into 259
merged(116, 32) into 260
merged(32, 97) into 261
merged(100, 32) into 262
merged(105, 110) into 263
merged(110, 32) into 264
merged(32, 105) into 265
merged(121, 32) into 266
merged(101, 114) into 267
merged(97, 110) into 268
merged(32, 115) into 269
merged(104, 256) into 270
merged(114, 32) into 271
merged(32, 119) into 272
merged(105, 115) into 273
merged(97, 116) into 274
merged(111, 32) into 275
merged(105, 116) into 276
merged(111, 110) into 277
merged(110, 100) into 278
merged(32, 111) into 279
merged(104, 97) into 280
merged(101, 110) into 281
merged(111, 114) into 282
merged(110, 103) into 283
merged(32, 98) into 284
merged(115, 116) into 285
merged(101, 115) into 286
merged(116, 111) into 287
merged(111, 117) into 288
merged(114, 101) into 289
merged(32, 109) into 290
merged(104, 101) into 291
merged(32, 99) into 292
merged(103, 32) into 293
merged(32, 104) into 294
merged(32, 

In [15]:
def encode(text, merges):
    
    # Convert text to bytes (ints 0–255)
    tokens = list(text.encode("utf-8"))

    i = 0
    while i < len(tokens) - 1:
        pair = (tokens[i], tokens[i + 1])
        if pair in merges:
            # Merge the pair
            tokens[i] = merges[pair]
            tokens.pop(i + 1)
            # Step back to check for overlapping merges
            if i > 0:
                i -= 1
        else:
            i += 1

    return tokens


In [16]:
sample_sentences= ["I love this movie",
         "This film was terrible...",
        "Absolutely fantastic experience.",
        "Not my cup of tea.",
        "The plot was predictable.",
        "Great acting and visuals.",
        "I would not recommend it.",
        "A masterpiece in storytelling.",
        "Poor dialogue, but good music.",
        "Loved every moment of it!"]
tokenized_samples=[encode(sentence, merges) for sentence in sample_sentences]
for i, tokens in enumerate(tokenized_samples):
    print(f"sentence-{i+1} : {tokens}")

sentence-1 : [73, 1456, 322, 576, 719, 1853, 381]
sentence-2 : [84, 909, 1155, 2412, 1285, 983, 3067, 439, 101, 46, 46, 46]
sentence-3 : [65, 616, 4115, 1687, 1172, 1180, 1195, 2058, 469, 2404, 381, 3056, 46]
sentence-4 : [78, 1302, 2565, 522, 4175, 1299, 304, 46]
sentence-5 : [84, 291, 2249, 1302, 1285, 1504, 2515, 362, 3058, 101, 46]
sentence-6 : [71, 1043, 751, 2160, 762, 718, 3062, 435, 1733, 46]
sentence-7 : [73, 1071, 1804, 1417, 1127, 371, 3375, 1229, 772, 46]
sentence-8 : [65, 1035, 1412, 618, 4079, 2259, 986, 1962, 1687, 2152, 103, 46]
sentence-9 : [80, 4040, 1528, 300, 537, 573, 44, 1057, 1143, 2265, 1728, 3409, 46]
sentence-10 : [76, 2786, 1427, 1135, 1430, 1978, 1173, 913, 276, 33]


In [17]:
import json
vocab = set(range(256))   #{0,1,2,3,...,255}
vocab.update(merges.values())# Add all new tokens created during merges
#vocab={0,1,2,...255,256,257....}

#Let's Sort for readability
vocab_list = sorted(vocab)

with open("subword_vocab.json", "w") as f:
    json.dump(vocab_list, f)

print(f"Saved vocabulary with {len(vocab_list)} tokens to subword_vocab.json")



Saved vocabulary with 4230 tokens to subword_vocab.json


In [18]:
#Fasttext and TF-IDF and accept only string tokens
sentences_tokenized = [
    [str(tok) for tok in encode(text,merges)]
    for text in X_train_clean
]


In [19]:
sentences_tokenized[:1]

[['718',
  '1266',
  '289',
  '806',
  '576',
  '495',
  '463',
  '890',
  '762',
  '557',
  '2151',
  '831',
  '2386',
  '2521',
  '271',
  '3068',
  '1643',
  '1044',
  '1313',
  '1730',
  '2740',
  '891',
  '4223',
  '1603',
  '3372',
  '576',
  '491',
  '753',
  '338',
  '2961',
  '513',
  '2782',
  '921',
  '1434',
  '701',
  '4120',
  '683',
  '2383',
  '665',
  '1387',
  '1101',
  '4031',
  '326',
  '449',
  '4083',
  '570',
  '874',
  '704',
  '653',
  '474',
  '436',
  '2254',
  '969',
  '2273',
  '515',
  '570',
  '2152',
  '380',
  '358',
  '2396',
  '653',
  '488',
  '2735',
  '267',
  '921',
  '2278',
  '2575',
  '4115',
  '355',
  '570',
  '341',
  '2407',
  '1009']]

In [20]:
from gensim.models import FastText

embedding_dim = 100

ft_model = FastText(
    sentences=sentences_tokenized,
    vector_size=embedding_dim,
    window=5,
    min_count=1, 
    workers=4,
    sg=1,
)

In [21]:
ft_model.wv.save_word2vec_format("embeddings.vec", binary=False)#binary=false implies save it in human readable format

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    tokenizer=lambda x: x,   #tokens already split,so no need to tokenize agin
    preprocessor=lambda x: x, #no need of preprocessing
    token_pattern=None
)

tfidf_matrix = tfidf.fit_transform(sentences_tokenized)
tfidf_vocab = tfidf.vocabulary_


In [23]:
import numpy as np

def tfidf_weighted_sentence_embedding(tokens, ft_model, tfidf_row, tfidf_vocab, dim):
    sent_vec = np.zeros(dim)
    weight_sum = 0.0

    for token in tokens:
        if token in ft_model.wv and token in tfidf_vocab:
            weight = tfidf_row[0, tfidf_vocab[token]]
            sent_vec += weight * ft_model.wv[token]
            weight_sum += weight

    if weight_sum > 0:
        sent_vec /= weight_sum

    return sent_vec


In [24]:
X_train_embeddings = np.array([
    tfidf_weighted_sentence_embedding(
        sentences_tokenized[i],
        ft_model,
        tfidf_matrix[i],
        tfidf_vocab,
        embedding_dim
    )
    for i in range(len(sentences_tokenized))
])


In [26]:
import numpy as np
#embedding_dim=model.vector_size

#word2vec return a model which contain A vocabulary embedding matrix
#def sentence_embedding(tokens, model):#this step is outside word2vec
   # vectors = [model.wv[str(t)] for t in tokens if str(t) in model.wv]

    #if len(vectors)==0:
        #return np.zeros(embedding_dim)
    #return np.mean(vectors, axis=0)


In [27]:
#X = []
y = df['sentiment']

#for text in tokenized_corpus:                
    #vec = sentence_embedding(text, model)
   # X.append(vec)

#X = np.array(X)

In [28]:
from sklearn.model_selection import train_test_split
X_train_new, X_test, y_train_new, y_test = train_test_split(X_train_embeddings , y , test_size=0.3, random_state=42)

In [36]:
y_train_new

52511    1
36485    1
48982    0
31737    1
16731    0
        ..
6265     1
54886    1
76820    0
860      1
15795    1
Name: sentiment, Length: 56000, dtype: int64

In [29]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_new, y_train_new)


LogisticRegression(max_iter=1000)

In [33]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_test,y_pred, average="weighted")

In [34]:
print("For custom tokenizer + TF-IDF-FASTTEXT+ LR")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

For custom tokenizer + TF-IDF-FASTTEXT+ LR
Accuracy: 0.6026666666666667
Precision: 0.602726762869952
Recall: 0.6026666666666667
F1-score: 0.6026478921491941


In [35]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier

clf = make_pipeline(
    StandardScaler(),
   SGDClassifier(
    loss="log_loss",
    max_iter=3000,
    alpha=1e-4,
    n_jobs=-1,
    early_stopping=True
)

)

clf.fit(X_train_new, y_train_new)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('sgdclassifier',
                 SGDClassifier(early_stopping=True, loss='log_loss',
                               max_iter=3000, n_jobs=-1))])

In [37]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

y_pred = clf.predict(X_test)

accuracy1 = accuracy_score(y_test, y_pred)
precision1, recall1, f11, _ = precision_recall_fscore_support(y_test,y_pred, average="weighted")

In [38]:
print("For custom tokenizer + TF-IDF-FASTTEXT+ SDG")
print("Accuracy:", accuracy1)
print("Precision:", precision1)
print("Recall:", recall1)
print("F1-score:", f11)

For custom tokenizer + TF-IDF-FASTTEXT+ SDG
Accuracy: 0.5585833333333333
Precision: 0.5585809821122778
Recall: 0.5585833333333333
F1-score: 0.5585818435219146


In [39]:
#main result 
#For custom tokenizer + TF-IDF-FASTTEXT+ LR
#Accuracy: 0.6007916666666666
#precision: 0.6009132720258751
#Recall: 0.6007916666666666
#F1-score: 0.6007339700998817